In [ ]:
import pandas as pd

In [ ]:
import os
os.chdir(os.getcwd().replace("\\notebooks", ""))

In [ ]:
student_data = pd.read_csv('rec_sys_uni/datasets/data/student/student_data.csv')
# Convert 'appraisal_date' to datetime for comparison
student_data['appraisal_date'] = pd.to_datetime(student_data['appraisal_date'])

In [ ]:
# Select one student from 2022 which take courses only in 2022 and did not appear in previous years
new_students = student_data[student_data['appraisal_date'] > '2022-08-30']
old_students = student_data[student_data['appraisal_date'] < '2022-08-30']
new_students = new_students[~new_students['student_num'].isin(old_students['student_num'])]
new_students

In [ ]:
# Pick average student by grade and also count of courses
new_students = new_students.groupby('student_num').agg({'grade': 'mean', 'module_abb': 'count'})
new_students

In [ ]:
# Pick student
student = student_data[student_data['module_abb'] == "SCI2018"]
student = student[student['student_num'].isin(new_students.index)]
student

In [ ]:
student = student_data[student_data['student_num'] == 24759684]

In [ ]:
# remove dublicates
student = student.drop_duplicates(subset=['module_abb'], keep='last')
student

In [ ]:
"""
student_data : dictionary  {
                                                courses_taken:
                                                    {
                                                        course_id(String):
                                                                            {
                                                                                passed: boolean,
                                                                                grade: float,
                                                                                period: int or [from, to],
                                                                                year: int
                                                                            },
                                                        ...
                                                    }
                                                }
"""
student_json = {'courses_taken': {}}
for i in range(len(student)):
    module_abb = student['module_abb'].values[i]
    passed = student['pass'].values[i]
    grade = student['grade'].values[i]
    appraisal_date = pd.to_datetime(student['appraisal_date'].values[i])
    year = appraisal_date.year
    month = appraisal_date.month

    student_json['courses_taken'][module_abb] = {
        'passed': bool(passed == 1),
        'grade': float(grade),
        'year': year
    }

    if month == 11:
        period = 1
    elif month == 12:
        period = 2
    elif month == 3:
        period = 4
    elif month == 6:
        period = 5
    else:
        period = student['period'].values[i]  # Default case if month doesn't match

    student_json['courses_taken'][module_abb]['period'] = period

student_json

In [ ]:
student_json
# Save student data in json file
# Serializing json
import json

json_object = json.dumps(student_json, indent=4)
with open('student.json', 'w') as fp:
    fp.write(json_object)